In [1]:
from pathlib import Path
import cv2
import numpy as np
import torch
from sapien.core import Pose

from real_robot.utils.visualization import Visualizer
from real_robot.sensors.simsense_depth import SimsenseDepth
from active_zero2.models.cgi_stereo.cgi_stereo import CGI_Stereo

data_dir = Path("capture").resolve()
image_paths = [p for p in data_dir.glob("*_images.npz") if "unaligned" not in p.name]  # all aligned images

# ----- ActiveZero++ ----- #
#ckpt_path = '/rl_benchmark/activezero2/model.pth'
ckpt_path = '/rl_benchmark/activezero2/model_oct19_veryclose.pth'
img_resize = (424, 240)
device = 'cuda:0'
disp_conf_topk = 2
disp_conf_thres = 0.8 # 0.95
MAX_DISP = 256

model = CGI_Stereo(maxdisp=MAX_DISP)
model.load_state_dict(torch.load(ckpt_path)['model'])
model = model.to(device)

def preprocess_image(image: np.ndarray) -> torch.Tensor:
    img_L = cv2.resize((image / 255.0).astype(np.float32), img_resize, interpolation=cv2.INTER_CUBIC)
    return torch.from_numpy(img_L).to(device)[None, None, ...]  # [1, 1, *img_resize]

# ----- Simsense ----- #
image_path = image_paths[0]
images = np.load(image_path)
params = np.load(image_path.parent / image_path.name.replace("images", "params"))

ir_shape = images["ir_l"].shape
assert images["ir_r"].shape == ir_shape, f'Mismatched IR shape: left={ir_shape}, right={images["ir_r"].shape}'
k_rgb = params["intrinsic_cv"]

k_irl = k_irr = np.array([[430.13980103,   0.        , 425.1628418 ],
                          [  0.        , 430.13980103, 235.27651978],
                          [  0.        ,   0.        ,   1.        ]])

# rsdevice.all_extrinsics["Infrared 1=>Infrared 2"]
T_irr_irl = np.array([[ 1.       ,  0.       ,  0.       , -0.0501572],
                      [ 0.       ,  1.       ,  0.       ,  0.       ],
                      [ 0.       ,  0.       ,  1.       ,  0.       ],
                      [ 0.       ,  0.       ,  0.       ,  1.       ]])
# rsdevice.all_extrinsics["Infrared 1=>Color"]
T_rgb_irl = np.array([[ 9.99862015e-01,  1.34780351e-02,  9.70994867e-03, 1.48976548e-02],
                      [-1.35059441e-02,  9.99904811e-01,  2.81448336e-03, 1.15314942e-05],
                      [-9.67109110e-03, -2.94523709e-03,  9.99948919e-01, 1.56505470e-04],
                      [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, 1.00000000e+00]])

depth_engine = SimsenseDepth(
    ir_shape[::-1], k_l=k_irl, k_r=k_irr, l2r=T_irr_irl, k_rgb=k_rgb,
    rgb_size=images["rgb"].shape[1::-1], l2rgb=T_rgb_irl,
    max_disp=1024, median_filter_size=3, depth_dilation=True
)
[p.stem for p in image_paths]

['60cm_images',
 '8cm_images',
 '45cm_max_power_images',
 '45cm_images',
 '8cm_max_power_images',
 '30cm_images',
 '15cm_max_power_images',
 '60cm_max_power_images',
 '30cm_max_power_images',
 '15cm_images']

In [2]:
obs_dict = {}

for image_path in image_paths:
    images = np.load(image_path)
    params = np.load(image_path.parent / image_path.name.replace("images", "params"))

    tag = image_path.stem.replace("_images", "")
    pose_world_camCV = Pose.from_transformation_matrix(params["cam2world_cv"])

    # RS capture
    obs_dict[f"vis_rs|{tag}_hand_camera_color"] = images["rgb"]
    obs_dict[f"vis_rs|{tag}_hand_camera_depth"] = images["depth"]
    obs_dict[f"vis_rs|{tag}_hand_camera_intr"] = params["intrinsic_cv"]
    obs_dict[f"vis_rs|{tag}_hand_camera_infrared_1"] = images["ir_l"]
    obs_dict[f"vis_rs|{tag}_hand_camera_infrared_2"] = images["ir_r"]
    obs_dict[f"vis_rs|{tag}_hand_camera_pose"] = pose_world_camCV

    # ----- ActiveZero++ ----- #
    orig_h, orig_w = images["ir_l"].shape
    with torch.no_grad():
        pred_dict = model({'img_l': preprocess_image(images["ir_l"]),
                           'img_r': preprocess_image(images["ir_r"])})
        torch.cuda.synchronize()
    for k in pred_dict:
        pred_dict[k] = pred_dict[k].detach().cpu().numpy()

    disparity = pred_dict['pred_orig'] # [1, H, W]
    disparity = disparity.squeeze() # [H, W]
    disparity_probs = pred_dict['cost_prob'].squeeze() # [1, disp//4, H, W]
    top_disparity_prob_idx = np.argpartition(-disparity_probs, disp_conf_topk, axis=0)[:disp_conf_topk, :, :]
    disparity_confidence = np.take_along_axis(disparity_probs, top_disparity_prob_idx, axis=0).sum(axis=0) # [H, W]
    disparity_conf_mask = disparity_confidence > disp_conf_thres

    # disparity => depth
    focal_length = k_irl[0, 0] * img_resize[0] / orig_w
    baseline = np.linalg.norm(T_irr_irl[:3, -1])
    depth = focal_length * baseline / (disparity + 1e-5)
    # filter out depth
    depth[~disparity_conf_mask] = 0.0

    # Upsample predicted depth image
    depth = cv2.resize(depth, images["ir_l"].shape[::-1], interpolation=cv2.INTER_NEAREST_EXACT)
    from real_robot.utils.camera import register_depth
    depth = register_depth(depth, k_irl, k_rgb, T_rgb_irl, images["rgb"].shape[1::-1], depth_dilation=True)
    # ActiveZero++ results
    obs_dict[f"vis_activezero2|{tag}_hand_camera_color"] = images["rgb"]
    obs_dict[f"vis_activezero2|{tag}_hand_camera_depth"] = depth
    obs_dict[f"vis_activezero2|{tag}_hand_camera_intr"] = params["intrinsic_cv"]
    obs_dict[f"vis_activezero2|{tag}_hand_camera_pose"] = pose_world_camCV

    # No upsample
    # resize_trans = np.array([[img_resize[0] / orig_w, 0, 0],
    #                          [0, img_resize[1] / orig_h, 0],
    #                          [0, 0, 1]], dtype=params["intrinsic_cv"].dtype)
    # # ActiveZero++ results
    # obs_dict[f"vis_activezero2|{tag}_hand_camera_color"] = cv2.resize(images["rgb"], img_resize, interpolation=cv2.INTER_CUBIC)
    # obs_dict[f"vis_activezero2|{tag}_hand_camera_depth"] = depth
    # obs_dict[f"vis_activezero2|{tag}_hand_camera_intr"] = resize_trans @ params["intrinsic_cv"]
    # obs_dict[f"vis_activezero2|{tag}_hand_camera_pose"] = pose_world_camCV

    # ----- Simsense ----- #
    depth_simsense = depth_engine.compute(images["ir_l"], images["ir_r"])
    # SimSense results
    obs_dict[f"vis_simsense|{tag}_hand_camera_color"] = images["rgb"]
    obs_dict[f"vis_simsense|{tag}_hand_camera_depth"] = depth_simsense
    obs_dict[f"vis_simsense|{tag}_hand_camera_intr"] = params["intrinsic_cv"]
    obs_dict[f"vis_simsense|{tag}_hand_camera_pose"] = pose_world_camCV

obs_dict = dict(sorted(obs_dict.items()))  # sort by key

visualizer = Visualizer(run_as_process=True)
visualizer.show_obs(obs_dict)
visualizer.render()

list(obs_dict.keys())

[2023-10-20 17:47:15,471] [CV2Visualizer] [cv2_visualizer.py:162] [INFO] Running <CV2Visualizer: Images> as a separate process


FEngine (64 bits) created at 0x55628956ad10 (threading is enabled)
FEngine resolved backend: OpenGL


[2023-10-20 17:47:17,745] [O3DGUIVisualizer] [o3d_gui_visualizer.py:1579] [INFO] Running <O3DGUIVisualizer: Point Clouds> as a separate process


['vis_activezero2|15cm_hand_camera_color',
 'vis_activezero2|15cm_hand_camera_depth',
 'vis_activezero2|15cm_hand_camera_intr',
 'vis_activezero2|15cm_hand_camera_pose',
 'vis_activezero2|15cm_max_power_hand_camera_color',
 'vis_activezero2|15cm_max_power_hand_camera_depth',
 'vis_activezero2|15cm_max_power_hand_camera_intr',
 'vis_activezero2|15cm_max_power_hand_camera_pose',
 'vis_activezero2|30cm_hand_camera_color',
 'vis_activezero2|30cm_hand_camera_depth',
 'vis_activezero2|30cm_hand_camera_intr',
 'vis_activezero2|30cm_hand_camera_pose',
 'vis_activezero2|30cm_max_power_hand_camera_color',
 'vis_activezero2|30cm_max_power_hand_camera_depth',
 'vis_activezero2|30cm_max_power_hand_camera_intr',
 'vis_activezero2|30cm_max_power_hand_camera_pose',
 'vis_activezero2|45cm_hand_camera_color',
 'vis_activezero2|45cm_hand_camera_depth',
 'vis_activezero2|45cm_hand_camera_intr',
 'vis_activezero2|45cm_hand_camera_pose',
 'vis_activezero2|45cm_max_power_hand_camera_color',
 'vis_activezero2

## Single capture

In [1]:
from pathlib import Path
import numpy as np
from real_robot.utils.visualization import Visualizer

from real_robot.sensors.simsense_depth import SimsenseDepth

data_dir = Path("capture").resolve()
tag = "15cm_max_power"
align_depth_to_color = True

gt_images = np.load(data_dir / "60cm_images.npz")
images = np.load(data_dir / f"{tag}_images.npz")
params = np.load(data_dir / f"{tag}_params.npz")
images_unaligned = np.load(data_dir / f"{tag}_unaligned_images.npz")
params_unaligned = np.load(data_dir / f"{tag}_unaligned_params.npz")
ir_shape = images["ir_l"].shape
assert images["ir_r"].shape == ir_shape, f'Mismatched IR shape: left={ir_shape}, right={images["ir_r"].shape}'
k_rgb = params["intrinsic_cv"]
k_irl = k_irr = np.array([[430.13980103,   0.        , 425.1628418 ],
                          [  0.        , 430.13980103, 235.27651978],
                          [  0.        ,   0.        ,   1.        ]])

# rsdevice.all_extrinsics["Infrared 1=>Infrared 2"]
T_irr_irl = np.array([[ 1.       ,  0.       ,  0.       , -0.0501572],
                      [ 0.       ,  1.       ,  0.       ,  0.       ],
                      [ 0.       ,  0.       ,  1.       ,  0.       ],
                      [ 0.       ,  0.       ,  0.       ,  1.       ]])
# rsdevice.all_extrinsics["Infrared 1=>Color"]
T_rgb_irl = np.array([[ 9.99862015e-01,  1.34780351e-02,  9.70994867e-03, 1.48976548e-02],
                      [-1.35059441e-02,  9.99904811e-01,  2.81448336e-03, 1.15314942e-05],
                      [-9.67109110e-03, -2.94523709e-03,  9.99948919e-01, 1.56505470e-04],
                      [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, 1.00000000e+00]])

if align_depth_to_color:
    depth_engine = SimsenseDepth(
        ir_shape[::-1], k_l=k_irl, k_r=k_irr, l2r=T_irr_irl, k_rgb=k_rgb,
        rgb_size=images["rgb"].shape[1::-1], l2rgb=T_rgb_irl,
        max_disp=1024, median_filter_size=3, depth_dilation=True
    )
else:
    depth_engine = SimsenseDepth(
        ir_shape[::-1], k_l=k_irl, k_r=k_irr, l2r=T_irr_irl, k_rgb=k_rgb,
        max_disp=1024, median_filter_size=3, depth_dilation=True
    )
    print("No align")

depth_simsense = depth_engine.compute(images["ir_l"], images["ir_r"])
depth_simsense.shape, depth_simsense.dtype

((480, 848), dtype('float32'))

In [2]:
from sapien.core import Pose
visualizer = Visualizer(run_as_process=True)
visualizer.show_obs({
    "vis_rs_hand_camera_color": images["rgb"],
    "vis_rs_hand_camera_depth": images["depth"],
    "vis_rs_hand_camera_intr": params["intrinsic_cv"],
    "vis_rs_hand_camera_infrared_1": images["ir_l"],
    "vis_rs_hand_camera_infrared_2": images["ir_r"],
    "vis_rs_hand_camera_pose": Pose.from_transformation_matrix(params["cam2world_cv"]),
    "vis_simsense_hand_camera_color": images["rgb"],
    "vis_simsense_hand_camera_depth": depth_simsense,
    "vis_simsense_hand_camera_intr": params["intrinsic_cv"],
    #"vis_simsense_hand_camera_intr": k_irl,
    "vis_simsense_hand_camera_pose": Pose.from_transformation_matrix(params["cam2world_cv"]),
})
visualizer.render()

[2023-10-20 15:47:23,105] [CV2Visualizer] [cv2_visualizer.py:162] [INFO] Running <CV2Visualizer: Images> as a separate process


FEngine (64 bits) created at 0x5595200ef9e0 (threading is enabled)
FEngine resolved backend: OpenGL


[2023-10-20 15:47:25,399] [O3DGUIVisualizer] [o3d_gui_visualizer.py:1579] [INFO] Running <O3DGUIVisualizer: Point Clouds> as a separate process


In [3]:
# Register depth (align depth to color)
from real_robot.utils.camera import register_depth

depth = register_depth(depth, k_irl, k_rgb, T_rgb_irl, images["rgb"].shape[1::-1], depth_dilation=True)
depth.shape, depth.dtype

((480, 848), dtype('float32'))

In [5]:
%timeit register_depth(depth, k_irl, k_rgb, T_rgb_irl, images["rgb"].shape[1::-1], depth_dilation=True)

6.16 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [2]:
# Register depth (align depth to color)
from real_robot.utils.camera import register_depth

depth = register_depth(depth_simsense, k_irl, k_rgb, T_rgb_irl, images["rgb"].shape[1::-1], depth_dilation=True)
depth.shape, depth.dtype

((480, 848), dtype('float32'))